In [1]:
!pip install --upgrade pip setuptools wheel -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [2]:
!pip install requests semanticscholar crossrefapi PyMuPDF -q


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.37.0 which is incompatible.


In [3]:
import json
import os
from datetime import datetime
import requests
from crossref.restful import Works
from semanticscholar import SemanticScholar
import fitz  # PyMuPDF


🔍 1. Search Semantic Scholar

In [4]:
def search_semantic_scholar(query, max_results=10):
    """
    Searches Semantic Scholar for research papers
    based on a topic input.
    Returns structured metadata for each paper.
    """
    print("🔍 Searching Semantic Scholar...")
    sch = SemanticScholar(timeout=10)

    papers = []
    try:
        # Perform API search
        results = sch.search_paper(query, limit=max_results)

        # Extract fields from each paper
        for p in results:
            papers.append({
                "title": p.title,
                "authors": [a.name for a in p.authors] if p.authors else [],
                "year": p.year,
                "pdf_url": p.openAccessPdf.get("url") if p.openAccessPdf else None,
                "citationCount": p.citationCount,
                "abstract": p.abstract,
                "url": p.url
            })
    except Exception as e:
        print("Semantic Scholar error:", e)

    return papers


In [5]:
def search_semantic_scholar(query, max_results=10):
    """
    Searches Semantic Scholar for research papers
    based on a topic input.
    Returns structured metadata for each paper.
    """
    print("🔍 Searching Semantic Scholar...")
    sch = SemanticScholar(timeout=10)

    papers = []
    try:
        # Perform API search
        results = sch.search_paper(query, limit=max_results)

        # Extract fields from each paper
        for p in results:
            papers.append({
                "title": p.title,
                "authors": [a.name for a in p.authors] if p.authors else [],
                "year": p.year,
                "pdf_url": p.openAccessPdf.get("url") if p.openAccessPdf else None,
                "citationCount": p.citationCount,
                "abstract": p.abstract,
                "url": p.url
            })
    except Exception as e:
        print("Semantic Scholar error:", e)

    return papers


📚 2. Search CrossRef (Backup API)

In [6]:
def search_crossref(query, max_results=5):
    """
    Searches CrossRef as a fallback API source.
    Useful when Semantic Scholar returns fewer results.
    """
    print("🔍 Searching CrossRef...")
    works = Works()
    papers = []

    try:
        # Search CrossRef and sort by relevance (score)
        results = works.query(query).sort("score")

        count = 0
        for item in results:
            if count >= max_results:
                break

            # Extract minimal metadata
            papers.append({
                "title": item.get("title", [""])[0],
                "authors": [a.get("family","") for a in item.get("author", [])],
                "year": item.get("issued", {}).get("date-parts", [[None]])[0][0],
                "pdf_url": None,            # CrossRef does not give PDFs
                "citationCount": None,
                "abstract": None,
                "url": item.get("URL", "")
            })
            count += 1

    except Exception as e:
        print("CrossRef error:", e)

    return papers


🧠 3. Main Function — Combine Both APIs & Save JSON

In [7]:
def run_module1(topic):
    """
    Runs the full Module-1 workflow:
    1. Search Semantic Scholar
    2. Search CrossRef
    3. Combine results
    4. Save output to JSON
    """
    print("\n===============================")
    print("MODULE 1: Paper Search Started")
    print("===============================\n")

    # API calls
    ss_results = search_semantic_scholar(topic, max_results=10)
    cr_results = search_crossref(topic, max_results=5)

    # Merge results
    all_papers = ss_results + cr_results

    print(f"\n📚 Total papers found: {len(all_papers)}")

    # Create output folder
    os.makedirs("data/search_results", exist_ok=True)

    # Dynamic filename
    filename = f"data/search_results/search_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

    # Save JSON data
    with open(filename, "w", encoding="utf-8") as f:
        json.dump({
            "topic": topic,
            "papers": all_papers
        }, f, indent=4, ensure_ascii=False)

    print("\n✅ Module 1 Completed!")
    print(f"📁 Results saved to: {filename}")

    return filename


▶ Run Module-1

In [8]:
topic = "AI system to automatically review and summarize research papers"
run_module1(topic)



MODULE 1: Paper Search Started

🔍 Searching Semantic Scholar...
🔍 Searching CrossRef...

📚 Total papers found: 16

✅ Module 1 Completed!
📁 Results saved to: data/search_results/search_results_20251224_092539.json


'data/search_results/search_results_20251224_092539.json'

🧩 MODULE 2 — PDF Filtering, Ranking, Downloading
Select PDFs → Download with Verification → Save Report

📥 1. Load Search Results

In [9]:
def load_search_results(filepath=None):
    """
    Loads the JSON file generated in Module-1.
    """
    if not filepath:
        print("❌ ERROR: Provide a valid filepath.")
        return None

    try:
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
        print(f" Loaded {len(data['papers'])} papers on topic: {data['topic']}")
        return data
    except Exception as e:
        print(f" Error loading search results: {e}")
        return None


📝 2. Filter Papers Having PDF Links

In [10]:
def filter_papers_with_pdfs(papers):
    """
    Selects only papers that have a valid PDF URL.
    """
    papers_with_pdf = [
        p for p in papers
        if p.get("pdf_url") and ("pdf" in p["pdf_url"].lower())
    ]

    print(f"\n PDF Check:")
    print(f" • Total papers: {len(papers)}")
    print(f" • Papers with PDF URLs: {len(papers_with_pdf)}")

    return papers_with_pdf


⭐ 3. Rank Papers (Citations + Year)

In [11]:
def rank_papers(papers):
    """
    Sorts papers by highest citation count and recent year.
    """
    valid = [
        p for p in papers
        if p.get("citationCount") is not None and p.get("year")
    ]

    ranked = sorted(valid, key=lambda x: (x["citationCount"], x["year"]), reverse=True)
    return ranked


🎯 4. Select Top N Papers

In [12]:
def select_top_papers(papers, count=3):
    """
    Selects the top N papers after:
    1. PDF filtering
    2. Ranking
    """
    papers_with_pdf = filter_papers_with_pdfs(papers)
    ranked = rank_papers(papers_with_pdf)
    selected = ranked[:count]

    print(f"\n Top {count} Selected Papers:")
    for i, p in enumerate(selected):
        print(f"\n{i+1}. {p['title']}")
        print(f"   ↳ Citations: {p['citationCount']}, Year: {p['year']}")

    return selected


 5. PDF Verification

In [13]:
def verify_pdf(filepath):
    """
    Checks if the file is a valid PDF using PyMuPDF.
    Size <1KB is marked invalid.
    """
    try:
        if not os.path.exists(filepath):
            return False
        if os.path.getsize(filepath) < 1024:
            return False
        with fitz.open(filepath) as doc:
            return len(doc) > 0
    except:
        return False


⬇ 6. Download PDF with Retry + Verification

In [14]:
def download_pdf_with_verification(url, filename, max_retries=3):
    """
    Downloads a PDF from a URL.
    Includes:
    - Retry logic
    - User-Agent header
    - PDF file validation
    """
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'Accept': 'application/pdf'
    }

    for attempt in range(max_retries):
        try:
            print(f"  Attempt {attempt + 1}/{max_retries}...")
            response = requests.get(url, headers=headers, timeout=30, stream=True)

            # Blocked access
            if response.status_code == 403:
                print("    HTTP 403 Forbidden. Retrying...")
                continue

            if response.status_code != 200:
                print(f"    HTTP Error: {response.status_code}")
                continue

            # Save PDF
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

            # Validate
            if verify_pdf(filename):
                print(f"    ✅ Downloaded: {os.path.getsize(filename):,} bytes")
                return True
            else:
                print("    ❌ Invalid PDF. Retrying...")
                os.remove(filename)

        except Exception as e:
            print(f"    Error: {str(e)[:50]}")

    return False


📩 7. Download All Selected Papers

In [15]:
def download_selected_papers(selected, output_dir="downloads"):
    """
    Downloads the top N research paper PDFs into a folder.
    """
    os.makedirs(output_dir, exist_ok=True)
    downloaded = []

    for i, paper in enumerate(selected):
        print(f"\n[{i+1}/{len(selected)}] Downloading:")
        print(" ", paper['title'])

        # Safe filename handling
        safe_title = "".join(c for c in paper['title'] if c.isalnum())[:30]
        filename = f"{output_dir}/{safe_title}.pdf"

        if download_pdf_with_verification(paper["pdf_url"], filename):
            print("   ✅ Success:", filename)
            paper["local_file"] = filename
            downloaded.append(paper)
        else:
            print("   ❌ Failed")

    return downloaded


📄 8. Save JSON Download Report

In [16]:
def save_download_report(downloaded, topic):
    """
    Creates a summary report of downloaded papers.
    """
    os.makedirs("data/reports", exist_ok=True)

    report = {
        "topic": topic,
        "download_count": len(downloaded),
        "papers": downloaded
    }

    output_file = f"data/reports/download_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(report, f, indent=4)

    print("\n Report saved to:", output_file)
    return output_file


🚀 9. Final Master Function (Module-2)

In [17]:
# =======================================================
# AUTO-LOAD LATEST SEARCH RESULTS
# =======================================================
import os
import glob

def get_latest_search_results():
    """
    Finds the most recent JSON file from data/search_results/
    """
    folder_path = "data/search_results/"
    json_files = glob.glob(os.path.join(folder_path, "*.json"))

    if not json_files:
        raise FileNotFoundError("❌ No search results found. Run Module 1 first.")

    latest_file = max(json_files, key=os.path.getctime)
    print(f"📄 Latest search results auto-loaded:\n{latest_file}")
    return latest_file


In [18]:
def main_module2(top_n=3):
    """
    MODULE 2 DRIVER FUNCTION
    ------------------------
    Auto-loads latest search results, selects top papers,
    downloads PDFs, and generates a report.
    """

    print("\n========== MODULE 2 STARTED ==========\n")

    # Step 1: Auto-load latest search_results file
    try:
        json_path = get_latest_search_results()
    except Exception as e:
        print(f"❌ ERROR: Failed to load latest search results.\n{e}")
        return None

    # Step 2: Load JSON content
    data = load_search_results(json_path)
    if not data:
        print("❌ ERROR: search_results JSON is empty or unreadable.")
        return None

    # Step 3: Filter & rank papers
    print("\n📊 Selecting top papers...\n")
    selected = select_top_papers(data["papers"], count=top_n)

    # Step 4: Download selected PDFs
    print("\n📥 Downloading PDFs...\n")
    downloaded = download_selected_papers(selected)

    # Step 5: Save results
    save_download_report(downloaded, data["topic"])

    print("\n========== MODULE 2 COMPLETED ==========\n")

    return downloaded


In [19]:
main_module2(top_n=3)



========== MODULE 2 STARTED ==========

📄 Latest search results auto-loaded:
data/search_results/search_results_20251224_092539.json
 Loaded 16 papers on topic: AI system to automatically review and summarize research papers

📊 Selecting top papers...


 PDF Check:
 • Total papers: 16
 • Papers with PDF URLs: 4

 Top 3 Selected Papers:

1. Automatic assessment of text-based responses in post-secondary education: A systematic review
   ↳ Citations: 92, Year: 2023

2. Editorial for Special Issue on Large-scale Pre-training: Data, Models, and Fine-tuning
   ↳ Citations: 2, Year: 2023

3. Special issue on future hybrid artificial intelligence and machine learning for smart expert systems
   ↳ Citations: 0, Year: 2021

📥 Downloading PDFs...


[1/3] Downloading:
  Automatic assessment of text-based responses in post-secondary education: A systematic review
  Attempt 1/3...
    ✅ Downloaded: 962,703 bytes
   ✅ Success: downloads/Automaticassessmentoftextbased.pdf

[2/3] Downloading:
  Editor

[{'title': 'Automatic assessment of text-based responses in post-secondary education: A systematic review',
  'authors': ['Rujun Gao',
   'H. Merzdorf',
   'S. Anwar',
   'M. C. Hipwell',
   'Arun Srinivasa'],
  'year': 2023,
  'pdf_url': 'https://arxiv.org/pdf/2308.16151',
  'citationCount': 92,
  'abstract': 'Text-based open-ended questions in academic formative and summative assessments help students become deep learners and prepare them to understand concepts for a subsequent conceptual assessment. However, grading text-based questions, especially in large courses, is tedious and time-consuming for instructors. Text processing models continue progressing with the rapid development of Artificial Intelligence (AI) tools and Natural Language Processing (NLP) algorithms. Especially after breakthroughs in Large Language Models (LLM), there is immense potential to automate rapid assessment and feedback of text-based responses in education. This systematic review adopts a scientific and r

In [20]:
# -*- coding: utf-8 -*-
"""
Milestone 2: PDF Text Extraction & Preprocessing

Automatically generated by Colab.
Enhanced version aligned with Infosys Springboard evaluation.

Author: Intern
Project: AI System to Automatically Review and Summarize Research Papers
"""

# ============================================================
# MODULE 0: INSTALL DEPENDENCIES
# ============================================================

!pip install pymupdf pymupdf4llm tqdm -q

# ============================================================
# MODULE 1: IMPORTS & CONFIGURATION
# ============================================================

import os
import re
import json
from pathlib import Path
from collections import Counter
from tqdm import tqdm
import fitz                    # PyMuPDF
import pymupdf4llm             # Layout-aware extraction

# Directory where PDFs were downloaded in Milestone 1
PDF_DIR = "downloads"

# Output directory for extracted structured data
OUTPUT_DIR = "data/extracted"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("📁 PDF Source Directory:", PDF_DIR)
print("📁 Output Directory:", OUTPUT_DIR)

# ============================================================
# MODULE 2: TEXT CLEANING UTILITIES
# ============================================================

def clean_text(text):
    """
    Cleans raw PDF text by:
    - Removing extra spaces
    - Fixing hyphenated line breaks
    - Removing unreadable characters
    """
    if not text:
        return ""

    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'-\s+', '', text)
    text = ''.join(c for c in text if ord(c) >= 32)

    return text.strip()

# ============================================================
# MODULE 3: PDF TEXT EXTRACTION
# ============================================================

def extract_text_from_pdf(pdf_path):
    """
    Extract text using two strategies:
    1) Layout-aware extraction (preferred)
    2) Standard PyMuPDF extraction (fallback)
    """

    print(f"\n📄 Processing: {pdf_path.name}")

    extracted_versions = []

    try:
        # Strategy 1: Layout-aware extraction
        try:
            layout_text = pymupdf4llm.to_markdown(str(pdf_path))
            if layout_text and len(layout_text) > 1000:
                extracted_versions.append(layout_text)
                print("   ✔ Layout-aware extraction used")
        except Exception:
            pass

        # Strategy 2: Standard PyMuPDF extraction
        doc = fitz.open(pdf_path)
        raw_text = ""
        for page in doc[:min(40, len(doc))]:
            raw_text += page.get_text()
        doc.close()

        if len(raw_text) > 1000:
            extracted_versions.append(raw_text)
            print("   ✔ Standard extraction used")

        if not extracted_versions:
            print("   ❌ No usable text extracted")
            return None

        best_text = max(extracted_versions, key=len)
        return clean_text(best_text)

    except Exception as e:
        print("   ❌ Extraction failed:", e)
        return None

# ============================================================
# MODULE 4: SECTION-WISE EXTRACTION
# ============================================================

def extract_sections(text):
    """
    Extract standard academic sections using regex.
    """

    sections = {
        "title": "",
        "abstract": "",
        "introduction": "",
        "methods": "",
        "results": "",
        "conclusion": ""
    }

    lines = text.split("\n")

    patterns = {
        "abstract": r"\babstract\b",
        "introduction": r"\bintroduction\b",
        "methods": r"\b(methods?|methodology|experiment)\b",
        "results": r"\b(results?|findings)\b",
        "conclusion": r"\b(conclusion|discussion)\b"
    }

    boundaries = {}

    for i, line in enumerate(lines):
        for section, pattern in patterns.items():
            if re.search(pattern, line.lower()):
                boundaries.setdefault(section, i)

    sorted_sections = sorted(boundaries.items(), key=lambda x: x[1])

    for idx, (section, start) in enumerate(sorted_sections):
        end = sorted_sections[idx + 1][1] if idx + 1 < len(sorted_sections) else len(lines)
        content = " ".join(lines[start:end]).strip()
        if len(content) > 200:
            sections[section] = content[:5000]

    # Extract title from first meaningful line
    for line in lines[:10]:
        if 20 < len(line) < 200:
            sections["title"] = line.strip()
            break

    return sections

# ============================================================
# MODULE 5: KEY FINDINGS EXTRACTION
# ============================================================

def extract_key_findings(sections, max_findings=5):
    """
    Extract key conclusions from Results and Conclusion sections.
    """

    keywords = [
        "we found", "results show", "our results",
        "significant", "improves", "outperforms",
        "demonstrates", "indicates", "we conclude"
    ]

    findings = []
    combined_text = sections.get("results", "") + " " + sections.get("conclusion", "")
    sentences = re.split(r'(?<=[.!?])\s+', combined_text)

    for sentence in sentences:
        if any(k in sentence.lower() for k in keywords):
            findings.append(sentence.strip())
        if len(findings) >= max_findings:
            break

    return findings

# ============================================================
# MODULE 6: VALIDATION CHECKS
# ============================================================

def validate_extraction(paper):
    """
    Validate extracted content quality.
    """
    return {
        "has_abstract": len(paper["sections"].get("abstract", "")) > 200,
        "has_multiple_sections": sum(len(v) > 200 for v in paper["sections"].values()) >= 2,
        "has_key_findings": len(paper["key_findings"]) > 0,
        "sufficient_text": paper["stats"]["total_chars"] > 1000
    }

# ============================================================
# MODULE 7: PROCESS SINGLE PDF
# ============================================================

def process_pdf(pdf_path):
    """
    End-to-end PDF processing.
    """
    text = extract_text_from_pdf(pdf_path)
    if not text:
        return None

    sections = extract_sections(text)
    key_findings = extract_key_findings(sections)

    paper = {
        "paper_id": pdf_path.stem,
        "filename": pdf_path.name,
        "sections": sections,
        "key_findings": key_findings,
        "stats": {
            "total_chars": len(text),
            "sections_found": sum(len(v) > 200 for v in sections.values())
        }
    }

    paper["validation"] = validate_extraction(paper)
    return paper

# ============================================================
# MODULE 8: CROSS-PAPER COMPARISON
# ============================================================

def compare_papers(papers):
    """
    Identify common themes across papers.
    """
    all_findings = []
    for paper in papers:
        all_findings.extend(paper["key_findings"])

    words = []
    for finding in all_findings:
        words.extend(re.findall(r'\b[a-zA-Z]{4,}\b', finding.lower()))

    return {
        "total_papers": len(papers),
        "total_findings": len(all_findings),
        "common_terms": Counter(words).most_common(10)
    }

# ============================================================
# MODULE 9: RUN MILESTONE 2 PIPELINE
# ============================================================

def run_milestone_2():
    pdf_files = list(Path(PDF_DIR).glob("*.pdf"))

    print("\n==============================")
    print(" MILESTONE 2 PIPELINE STARTED ")
    print("==============================")
    print(f"📄 PDFs Found: {len(pdf_files)}")

    if not pdf_files:
        print("❌ No PDFs found. Run Milestone 1 first.")
        return

    processed_papers = []

    for pdf in tqdm(pdf_files):
        paper = process_pdf(pdf)
        if paper:
            processed_papers.append(paper)
            with open(f"{OUTPUT_DIR}/{paper['paper_id']}.json", "w", encoding="utf-8") as f:
                json.dump(paper, f, indent=2)

    comparison = compare_papers(processed_papers)
    with open(f"{OUTPUT_DIR}/comparison_report.json", "w", encoding="utf-8") as f:
        json.dump(comparison, f, indent=2)

    print("\n✅ MILESTONE 2 COMPLETED SUCCESSFULLY")
    print(f"📊 Papers Processed: {len(processed_papers)}")
    print("📁 Output Saved To:", OUTPUT_DIR)

    return processed_papers, comparison

# ============================================================
# EXECUTION
# ============================================================

results, comparison = run_milestone_2()


Consider using the pymupdf_layout package for a greatly improved page layout analysis.
📁 PDF Source Directory: downloads
📁 Output Directory: data/extracted

 MILESTONE 2 PIPELINE STARTED 
📄 PDFs Found: 2


  0%|          | 0/2 [00:00<?, ?it/s]


📄 Processing: Automaticassessmentoftextbased.pdf


 50%|█████     | 1/2 [00:21<00:21, 21.97s/it]

   ✔ Layout-aware extraction used
   ✔ Standard extraction used

📄 Processing: EditorialforSpecialIssueonLarg.pdf


100%|██████████| 2/2 [00:22<00:00, 11.48s/it]

   ✔ Layout-aware extraction used
   ✔ Standard extraction used

✅ MILESTONE 2 COMPLETED SUCCESSFULLY
📊 Papers Processed: 2
📁 Output Saved To: data/extracted
